# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [3]:
school_data_complete.head(5)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [8]:
new_dist_sum = school_data_complete[['school_name','budget']]
new_dist_sum_u = new_dist_sum.drop_duplicates('school_name')

math_pass_pct = school_data_complete[['math_score','reading_score'] ]
math_pass_pct_m = math_pass_pct['math_score'] >= 70
math_pass_pct_e = math_pass_pct['reading_score'] >= 70
math_pass_pct_eng = school_data_complete[math_pass_pct_m]
math_pass_pct_math = school_data_complete[math_pass_pct_e]

district_summary = pd.DataFrame([[
                    len(school_data_complete['school_name'].unique()),
                    (len(school_data_complete['Student ID'].unique())),
                    ((new_dist_sum_u['budget'].sum())),
                    (school_data_complete['math_score'].mean()),
                    (school_data_complete['reading_score'].mean()),
                    (math_pass_pct_math['Student ID'].count() * 100/school_data_complete['Student ID'].count() ),
                    (math_pass_pct_eng['Student ID'].count() * 100/school_data_complete['Student ID'].count() ),
                    (((school_data_complete['math_score'].mean()) +
                    (school_data_complete['reading_score'].mean()))/2)
                    ]],
                    index=[0],
                    columns = ['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score'
                              ,'% Passing Math','% Passing Reading','% Overall Passing Rate'])
district_summary

#format cells
district_summary.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.6f}", 
                       "Average Math Score": "{:.6f}", 
                       "% Passing Math": "{:.6f}", 
                       "% Passing Reading": "{:.6f}", 
                       "% Overall Passing Rate": "{:.6f}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.877840,85.805463,74.980853,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [9]:
#groups by school
grpby_school = school_data_complete.set_index('school_name').groupby(['school_name'])

#school types
school_types = school_data.set_index('school_name')['type']

# total students by school
student_per_sch = grpby_school['Student ID'].count()

# school budget
school_budget = school_data.set_index('school_name')['budget']

#student budget
student_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

#avg scores by school
avg_math = grpby_school['math_score'].mean()
avg_read = grpby_school['reading_score'].mean()

# % passing scores
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student_per_sch 
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student_per_sch 

over_all = ((school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/student_per_sch ) +
          (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/student_per_sch ))/2  

school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": student_per_sch,
    "Per Student Budget": student_budget,
    "Total School Budget": school_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": over_all
})
#munging
school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#  formatting
school_summary.head().style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.6%}", 
                          "% Passing Reading": "{:.6%}", 
                          "Overall Passing Rate": "{:.6%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064%,81.933280%,74.306672%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,95.586652%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,73.363852%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602%,79.299014%,73.804308%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,95.265668%


In [10]:
# Top rated schools
top_schools = school_summary.sort_values("Overall Passing Rate", ascending = False)
top_schools.head().style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.6%}", 
                          "% Passing Reading": "{:.6%}", 
                          "Overall Passing Rate": "{:.6%}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,95.586652%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171%,97.308869%,95.290520%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595%,95.945946%,95.270270%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,95.265668%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718%,96.539641%,95.203679%


## Bottom Performing Schools (By Passing Rate)


* Sort and display the five worst-performing schools

In [11]:
# Bottom rated schools
bottom_schools = school_summary.sort_values("Overall Passing Rate", ascending = True)
bottom_schools.head().style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:,.2f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.6%}", 
                          "% Passing Reading": "{:.6%}", 
                          "Overall Passing Rate": "{:.6%}"})


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592%,80.220055%,73.293323%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,73.363852%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922%,81.316421%,73.500171%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551%,81.222432%,73.639992%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602%,79.299014%,73.804308%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [12]:
#create grade level average math scores for each school 
ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby("school_name")["math_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby("school_name")["math_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby("school_name")["math_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby("school_name")["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
del math_scores.index.name  

#show and format
math_scores.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})



,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [13]:
#create grade level average math scores for each school 
ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby("school_name")["reading_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby("school_name")["reading_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby("school_name")["reading_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby("school_name")["reading_score"].mean()

reading_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
del reading_scores.index.name  

#show and format
reading_scores.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})



,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [14]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 584.999, 614.999, 644.999, 999999]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [15]:
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], spending_bins, labels = group_names)

#group by spending
by_spending = school_data_complete.groupby('spending_bins')

#calculations
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
over_all = ((school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count())+(school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()))/2  
          
# df build            
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "% Overall Passing Rate": over_all           
})
            
#reorder columns
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]

scores_by_spend.index.name = "Spending Ranges (Per Student)"

#formating
scores_by_spend.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6%}', 
                              '% Passing Reading':'{:.6%}', 
                              '% Overall Passing Rate': '{:.6%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889%,96.686558%,95.194724%
$585-615,83.529196,83.838414,94.124128%,95.886889%,95.005509%
$615-645,78.061635,81.434088,71.400428%,83.614770%,77.507599%
$645-675,77.049297,81.005604,66.230813%,81.109397%,73.670105%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [16]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 999, 1999, 99999999999]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [17]:
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], size_bins, labels = group_names)

#group by spending
by_size = school_data_complete.groupby('size_bins')

#calculations 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
over_all = ((school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count())+(school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()))/2
          
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "% Overall Passing Rate": over_all            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]

scores_by_size.index.name = "School Size"

#formating
scores_by_size.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6%}', 
                              '% Passing Reading':'{:.6%}', 
                              '% Overall Passing Rate': '{:.6%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.828654,93.952484%,96.040317%,94.996400%
Medium (1000-2000),83.372682,83.372682,93.616522%,96.773058%,95.194790%
Large (2000-5000),77.477597,77.477597,68.652380%,82.125158%,75.388769%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [18]:
# group by type of school
by_type = school_data_complete.groupby("type")

#calculations 
avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
over_all = ((school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count())+(school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()))/2

# df build            
scores_by_type = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "% Overall Passing Rate": over_all})
    
#reorder columns
scores_by_type = scores_by_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]
scores_by_type.index.name = "School Type"


#formating
scores_by_type.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6%}', 
                              '% Passing Reading':'{:.6%}', 
                              '% Overall Passing Rate': '{:.6%}'})



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.406183,93.701821%,96.645891%,95.173856%
District,76.987026,76.987026,66.518387%,80.905249%,73.711818%
